## Is it timely

Meine Internet Recherche ergab, dass es einen aktuelleren Datensatz als den im Buch via GoogleDrive angebotenen geben sollte:
- `public_150k_plus_230930`
- Published: 30.09.2023
- Quelle: https://data.sba.gov/dataset/ppp-foia/resource/738e639c-1fbf-4e16-beb0-a223831011e8
- Datengröße 452 MB

Der vom Buch ist
- `public_150k_plus_recent`
- Datengröße 322 MB

## Is it Complete

### Earliest Date

In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
#print min and max data from book data
import pandas as pd

ppp_data_book = pd.read_csv('../data/public_150k_plus_recent.csv')

ppp_data_book['DateApproved'] = pd.to_datetime(ppp_data_book['DateApproved'], format='%m/%d/%Y')

print(ppp_data_book['DateApproved'].min())
print(ppp_data_book['DateApproved'].max())

2020-04-03 00:00:00
2021-01-31 00:00:00


In [3]:
#print min and max data from found data
import pandas as pd

ppp_data_found = pd.read_csv('../data/public_150k_plus_230930.csv')

ppp_data_book['DateApproved'] = pd.to_datetime(ppp_data_book['DateApproved'], format='%m/%d/%Y')

print(ppp_data_book['DateApproved'].min())
print(ppp_data_book['DateApproved'].max())

2020-04-03 00:00:00
2021-01-31 00:00:00


Beiden Datenfiles haben dasselbe früheste bzw. späteste Datum. In diesem Bezug sind sie gleich aktuell.

### File-Size / Row-Size

- Gefundene Datensatz 452MB > Datensatz Buch 322.5 MB und zwar um 129.5 MB

für Zeilenzahl gibts kurzes Script:

In [4]:
data_book = open('../data/public_150k_plus_recent.csv', 'r')
data_found = open('../data/public_150k_plus_230930.csv', 'r')

lines_book = len(data_book.readlines())
lines_found = len(data_found.readlines())

print(f"Book lines: {lines_book}")
print(f"Found lines: {lines_found}")
print(f"Gefundene Datensatz hat um: {lines_found - lines_book} mehr Zeilen")

Book lines: 766500
Found lines: 968526
Gefundene Datensatz hat um: 202026 mehr Zeilen


In diesem Bezug ist der gefundene Datensatz also aktueller

### Inhaltsvergleich der beiden Dateien

In [5]:
# quick script for creating new CSVs that each contain the first few rows of our larger data files

import pandas as pd

ppp_data_book = pd.read_csv('../data/public_150k_plus_recent.csv')
ppp_data_found = pd.read_csv('../data/public_150k_plus_230930.csv')

book_sample = ppp_data_book.head()
found_sample = ppp_data_found.head()

book_sample.to_csv('../data/book_sample.csv', index=False)
found_sample.to_csv('../data/found_sample.csv', index=False)

Gefundenen sind folgende Felder ausgefüllt, welche bei Buch Daten nicht ausgefüllt waren
- LoanStatusDate
- LoanSatatus war bei allen Paid in Full
- LMIIndicator

Neue Datei hat 2 neue Spalten
- ForgivnessAmount
- ForgivnessDate

Merge der beiden Dateien um Unterschiede zu identifizieren

In [1]:
import pandas as pd

book_ppp_data = pd.read_csv('../data/public_150k_plus_recent.csv')
found_ppp_data = pd.read_csv('../data/public_150k_plus_230930.csv')

merged_data = pd.merge(book_ppp_data, found_ppp_data, how='outer', 
                       left_on=['BorrowerName'], 
                       right_on=['BorrowerName'], indicator=True)

print(merged_data.value_counts('_merge'))

_merge
both          894727
right_only    159139
left_only      21299
dtype: int64


mit 'DateApproved' / ohne 'DateApproved' / nur 'BorrowerName'
- both: 635611 / 736709 / 894722
- right_only 333477 / 284830 / 159139
- left_only 131432 / 130418 / 21299

In [2]:
21299/893722

0.023831795569539523

2% der Daten sind `verschwunden`. Hier sollte man beginnen nachzufragen, was ist passiert ist der Kredit nicht durchgegangen etc....

## Is it Well-Annotated

- try to find a data dictionary

In [3]:
# quick script for reviewing all the column names in the PPP data to see what we can infeer abut them from
# the data itself

import pandas as pd

ppp_data_sample = pd.read_csv('../data/found_sample.csv')

# convert all missing data entries to '<NA>' sing the convertdtypes() method
converted_data_sample = ppp_data_sample.convert_dtypes()

transposed_ppp_data_sample = converted_data_sample.transpose()

print(transposed_ppp_data_sample)

                                                             0  \
LoanNumber                                          9547507704   
DateApproved                                        05/01/2020   
SBAOfficeCode                                              464   
ProcessingMethod                                           PPP   
BorrowerName                             SUMTER COATINGS, INC.   
BorrowerAddress                          2410 Highway 15 South   
BorrowerCity                                            Sumter   
BorrowerState                                             <NA>   
BorrowerZip                                         29150-9662   
LoanStatusDate                                      12/18/2020   
LoanStatus                                        Paid in Full   
Term                                                        24   
SBAGuarantyPercentage                                      100   
InitialApprovalAmount                                769358.78   
CurrentApp

## Is it high volume
- es ist klar das viele Zeilen vorhanden sind
- nicht klar ist wieviele Datenfelder tatsächlich ausgefüllt worden sind

In [4]:
# quick script for reviewing all the column names in the PPP data to see what we can
# infer about them from the data itself

import pandas as pd

ppp_data = pd.read_csv('../data/public_150k_plus_230930.csv')

print(ppp_data.value_counts('LoanStatus'))
print(sum(ppp_data.value_counts('LoanStatus')))

print()
print(ppp_data.value_counts('Gender'))
print(sum(ppp_data.value_counts('Gender')))

print()
print(ppp_data['BorrowerAddress'].isna().sum())

LoanStatus
Paid in Full    937553
Charged Off      15879
Exemption 4      15093
dtype: int64
968525

Gender
Unanswered      579889
Male Owned      315819
Female Owned     72817
dtype: int64
968525

14


### Is it Consistent?

In [2]:
# quick script for finding the minimum and maximum loans currently approved in our PPP loan dataset

import pandas as pd

ppp_data = pd.read_csv('../data/public_150k_plus_230930.csv')
print(ppp_data['CurrentApprovalAmount'].min())
print(ppp_data['CurrentApprovalAmount'].max())

150000.0
10000000.0


check for spelling issues

Fingerprinting (clustering)
1. Removes leading and trailing whitespaces
2. Changes all characters to their lowercase representation
3. Remove all puntuation and control chartacters
4. Normalized extended Western characters to their ASCII representation 'gödel' => 'godel'
5. Splits the string into whitespace-seperated tokens
6. Sorts the tokens and removes duplicates
7. Join the tokens back together

In [3]:
pip install fingerprints

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 5.1 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# quick scipt for determining wheter there are typos &c. in any of the PPP loan data's bank names

import pandas as pd
import fingerprints

ppp_data = pd.read_csv('../data/public_150k_plus_230930.csv')

unique_names = ppp_data['OriginatingLender'].unique()
print(len(unique_names))

fingerprint_list = []
for name in unique_names:
    fingerprint_list.append(fingerprints.generate(name))
    
fingerprint_set = set(fingerprint_list)
print(len(fingerprint_set))

4236
4137


/home/pyspark-client/.local/lib/python3.10/site-packages/fingerprints/cleanup.py:66: ICUWarning: Install 'pyicu' for better text transliteration.
  text = ascii_text(text)


### Is it multi-variant

wieviele spalten gibt es, welche davon enthalten einen Wert

In [11]:
# quick script for determining what borrowers did (or really, did not) state they would use PPP loan funds for
print(ppp_data['UTILITIES_PROCEED'].isna().sum())
print(ppp_data['PAYROLL_PROCEED'].isna().sum())
print(ppp_data['MORTGAGE_INTEREST_PROCEED'].isna().sum())
print(ppp_data['RENT_PROCEED'].isna().sum())
print(ppp_data['REFINANCE_EIDL_PROCEED'].isna().sum())
print(ppp_data['HEALTH_CARE_PROCEED'].isna().sum())
print(ppp_data['DEBT_INTEREST_PROCEED'].isna().sum())

payroll_only = ppp_data[
    (ppp_data['UTILITIES_PROCEED'].isna()) & 
    (ppp_data['MORTGAGE_INTEREST_PROCEED'].isna()) &
    (ppp_data['RENT_PROCEED'].isna()) &
    (ppp_data['REFINANCE_EIDL_PROCEED'].isna()) &
    (ppp_data['HEALTH_CARE_PROCEED'].isna()) &
    (ppp_data['DEBT_INTEREST_PROCEED'].isna())
]

print(len(payroll_only.index))

629156
1832
922353
868995
945677
911084
936818
597118


## Assessing Data Fit
### Validity


In [13]:
# determining how many loans have been disbured
print(ppp_data['LoanStatus'].value_counts())
print(sum(ppp_data['LoanStatus'].value_counts()))

Paid in Full    937553
Charged Off      15879
Exemption 4      15093
Name: LoanStatus, dtype: int64
968525
